In [4]:
import tensorflow as tf
import numpy as np

%matplotlib inline

'''
Класс, представляющий собой одну кинематическую пару
'''
class KinematicPart:
    s = tf.constant(0, dtype=tf.float32)
    a = tf.constant(0, dtype=tf.float32)
    alpha = tf.constant(0, dtype=tf.float32)
    
    def __init__(self, s,a,alpha):
        self.s = tf.constant(s, dtype=tf.float32)
        self.a = tf.constant(a, dtype=tf.float32)
        self.alpha = tf.constant(alpha, dtype=tf.float32)
        
    def getMatrix(self, q):
        return [
            [tf.cos(q), -tf.sin(q)*tf.cos(self.alpha), tf.sin(q)*tf.sin(self.alpha), self.a*tf.cos(q)],
            [tf.sin(q), tf.cos(q)*tf.cos(self.alpha), -tf.cos(q)*tf.sin(self.alpha), self.a*tf.sin(q)],
            [0, tf.sin(self.alpha), tf.cos(self.alpha), self.s],
            [0, 0, 0, 1]
                         ]


'''
Класс робота, состоящего из пар
'''
class Robot:
    parts = []
    
    def __init__(self, parts):
        self.parts = parts
     
    '''
    Получить координаты схвата (конечного звена)
    '''
    def getXYZ(self, Q):
        return self.getXYZPair(Q, len(self.parts))
    
    '''
    Получить координаты конкретной пары 
    '''    
    def getXYZPair(self, Q, pair):
        
        resultMatrix = tf.eye(4, dtype=tf.float32)
        
        for i,p in enumerate(self.parts):
            
            if i==pair:
                break
            
            resultMatrix = tf.matmul(resultMatrix,p.getMatrix(Q[i]))
        
        xyz1 = tf.matmul(resultMatrix,tf.constant([[0],[0],[0],[1]], dtype=tf.float32))
        #res = np.squeeze(xyz1,[3])
        return xyz1
    
    
    '''
    Массив координат всех пар (для построения графика)
    '''
    def getPairPoints(self, Q):
        
        result = []
        
        for i,p in enumerate(self.parts):
            pairXYZ = self.getXYZPair(Q,i)
            result.append([pairXYZ[0],pairXYZ[1], pairXYZ[2]])
        
        return result


In [12]:
r=np.pi/180.0;
Z1 = KinematicPart(450, 320, np.pi/2)
Z2 = KinematicPart(0, 870, 0)
Z3 = KinematicPart(0, 1015, 0)
Z4 = KinematicPart(0, 0, np.pi/2)
Z5 = KinematicPart(0, 235, 0)

parts = [Z1,Z2,Z3,Z4,Z5]

kuka = Robot(parts)

Q = tf.Variable([  0,   70*r,   230*r,
         120*r,   0], dtype=tf.float32)

xyz = kuka.getXYZ(Q)[:3]

In [11]:
model = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(model)
    print(session.run(txyz))

[[ 1022.55761719]
 [ 1230.        ]
 [  370.03283691]]


In [48]:
Q0 = tf.Variable([  0,   20*r,   20*r,
         20*r,   0],dtype=tf.float32)

target = tf.constant([[900],[1000],[222]], dtype=tf.float32)

xyz = kuka.getXYZ(Q0)[:3]

loss = tf.reduce_sum(tf.sqrt(tf.pow(target-xyz, 2)))

optimizer = tf.train.RMSPropOptimizer(0.05)

train = optimizer.minimize(loss)

model = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(model)
    bestQ = session.run(Q0)
    bestLoss = session.run(loss)
    print("starting at", "Q:", bestQ, "loss:", bestLoss)
    for step in range(5000):  
        session.run(train)
        
        curQ = session.run(Q0)
        curLoss = session.run(loss)
        
        if curLoss < bestLoss:
            bestLoss = curLoss
            bestQ = curQ
        
        if(step % 50 == 0):
            print("step", step, "loss:", bestQ, "loss:", bestLoss)
    
    print("bestQ: ", bestQ, " bestLoss: ", bestLoss)

starting at Q: [ 0.          0.34906584  0.34906584  0.34906584  0.        ] loss: 3514.07
step 0 loss: [ 0.15811369  0.19095424  0.19141556  0.50708365 -0.15810099] loss: 2887.39
step 50 loss: [ 0.44634253  0.00786    -0.18939403  0.64486682 -2.05218387] loss: 989.65
step 100 loss: [ 0.64476562  0.19616881 -0.81027079  0.20415816 -3.06229806] loss: 701.285
step 150 loss: [ 0.80672216  0.57842249 -1.64448833  0.36607713 -3.08528376] loss: 73.9481
step 200 loss: [ 0.82425773  0.62020767 -1.6915679   0.33558872 -3.06608462] loss: 6.93327
step 250 loss: [ 0.82660878  0.61751044 -1.69165885  0.31441164 -3.08559656] loss: 4.26808
step 300 loss: [ 0.82660878  0.61751044 -1.69165885  0.31441164 -3.08559656] loss: 4.26808
step 350 loss: [ 0.82660878  0.61751044 -1.69165885  0.31441164 -3.08559656] loss: 4.26808
step 400 loss: [ 0.82660878  0.61751044 -1.69165885  0.31441164 -3.08559656] loss: 4.26808
step 450 loss: [ 0.82660878  0.61751044 -1.69165885  0.31441164 -3.08559656] loss: 4.26808
ste